In [36]:
from numpy import loadtxt
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense 
import pandas as pd 
import numpy as np
from sklearn.model_selection import GridSearchCV
from tensorflow.python.keras.wrappers.scikit_learn import KerasClassifier
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import SGD

In [2]:
data = loadtxt('pima-indians-diabetes.csv', delimiter=',')

In [3]:
X = data[:, 0:8]
y = data[:, 8]

In [4]:
ann_model = Sequential()
ann_model.add(Dense(12, input_dim=8, activation='relu'))
ann_model.add(Dense(8, activation='relu'))
ann_model.add(Dense(1, activation='sigmoid'))

In [5]:
ann_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [6]:
ann_model.fit(X, y, epochs=150, batch_size=10, verbose=0)

In [7]:
_, accuracy = ann_model.evaluate(X,y)
print('Accuracy: %.2f' % (accuracy*100))

24/24 [==============================] - 1s 5ms/step - loss: 0.4517 - accuracy: 0.7891
Accuracy: 78.91


In [8]:
predictions = ann_model.predict(X)

In [9]:
for i in range(5):
    print('%s => %d (expected %d)' % (X[i].tolist(), predictions[i], y[i]))

[6.0, 148.0, 72.0, 35.0, 0.0, 33.6, 0.627, 50.0] => 0 (expected 1)
[1.0, 85.0, 66.0, 29.0, 0.0, 26.6, 0.351, 31.0] => 0 (expected 0)
[8.0, 183.0, 64.0, 0.0, 0.0, 23.3, 0.672, 32.0] => 0 (expected 1)
[1.0, 89.0, 66.0, 23.0, 94.0, 28.1, 0.167, 21.0] => 0 (expected 0)
[0.0, 137.0, 40.0, 35.0, 168.0, 43.1, 2.288, 33.0] => 0 (expected 1)


TUNING THE MODEL TO IMPROVE ACCURACY AND EFFICIENCY
Tuning for the Batch Size and the Number of epochs

In [11]:
def create_model():
    model = Sequential()
    model.add(Dense(12, input_dim=8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [14]:

seed=7
np.random.seed(seed)

In [16]:
model = KerasClassifier(build_fn=create_model, verbose=0)

In [18]:
batch = [10, 20, 40, 60, 80, 100]
epoch = [10, 50, 100]
parameter_grid = dict(batch_size=batch, epochs=epoch)
grid = GridSearchCV(estimator=model, param_grid=parameter_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, y)

In [19]:
print('Best %f using %s' % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param, in zip(means, stds, params):
    print('%f (%f) with: %r' % (mean, stdev, param))

Best 0.695312 using {'batch_size': 10, 'epochs': 100}
0.583333 (0.013279) with: {'batch_size': 10, 'epochs': 10}
0.675781 (0.011500) with: {'batch_size': 10, 'epochs': 50}
0.695312 (0.005524) with: {'batch_size': 10, 'epochs': 100}
0.593750 (0.041707) with: {'batch_size': 20, 'epochs': 10}
0.667969 (0.038273) with: {'batch_size': 20, 'epochs': 50}
0.692708 (0.032578) with: {'batch_size': 20, 'epochs': 100}
0.588542 (0.067079) with: {'batch_size': 40, 'epochs': 10}
0.645833 (0.039879) with: {'batch_size': 40, 'epochs': 50}
0.665365 (0.018688) with: {'batch_size': 40, 'epochs': 100}
0.518229 (0.078407) with: {'batch_size': 60, 'epochs': 10}
0.611979 (0.016367) with: {'batch_size': 60, 'epochs': 50}
0.654948 (0.040386) with: {'batch_size': 60, 'epochs': 100}
0.477865 (0.040133) with: {'batch_size': 80, 'epochs': 10}
0.619792 (0.014382) with: {'batch_size': 80, 'epochs': 50}
0.664062 (0.014616) with: {'batch_size': 80, 'epochs': 100}
0.592448 (0.068875) with: {'batch_size': 100, 'epochs': 

In [31]:
def create_model_1(optimizer='adam'):
    model_1 = Sequential()
    model_1.add(Dense(12, input_dim=8, activation='relu'))
    model_1.add(Dense(1, activation='sigmoid'))

    model_1.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model_1

Tuning the optimization algorithm 

In [32]:
model_1_1 = KerasClassifier(build_fn=create_model_1, epochs=100, batch_size=10, verbose=0)

In [33]:
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=model_1_1, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, y)

In [34]:
print('Best: %f using %s' % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print('%f (%f) with %r' % (mean, stdev, param))

Best: 0.696615 using {'optimizer': 'Adam'}
0.661458 (0.023939) with {'optimizer': 'SGD'}
0.647135 (0.019488) with {'optimizer': 'RMSprop'}
0.631510 (0.026748) with {'optimizer': 'Adagrad'}
0.570312 (0.132927) with {'optimizer': 'Adadelta'}
0.696615 (0.038051) with {'optimizer': 'Adam'}
0.687500 (0.019137) with {'optimizer': 'Adamax'}
0.675781 (0.024080) with {'optimizer': 'Nadam'}


Tuning the Learning rate and the Momentum

In [42]:

def create_model_2(learn_rate=0.01, momentum=0):
    model_2 = Sequential()
    model_2.add(Dense(12, input_dim=8, activation='relu'))
    model_2.add(Dense(1, activation='sigmoid'))

    optimizer = SGD(lr=learn_rate, momentum=momentum)

    model_2.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model_2

In [38]:
model_2_2 = KerasClassifier(build_fn=create_model_2, epochs=100, batch_size=10, verbose=0)

In [39]:
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]

In [43]:
param_grid= dict(learn_rate=learn_rate, momentum=momentum)
grid = GridSearchCV(estimator=model_2_2, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, y)

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_resultss_['params']

for mean, stdev, param in zip (means, stds, params):
    print('%f with: %r' % (mean, stdev, param))